# Score compression of rescaled data (subsamples) with subsequent neural density estimation

In [1]:
import numpy as np
import sys
# load your pydelfi version here (not necessary to include path if you have pip installed)
sys.path.append("/home/nessa/Documents/Projects/pydelfi/") 
import ndes.ndes as ndes
import delfi.delfi as delfi
import compression.score.score as score
import distributions.priors as priors
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
%matplotlib inline

### load the data

In [2]:
# load data (note that order of params is not the same as data_sim)
data_scaled        = np.load('./data/data_full_set_scaled.npy')
data_scaled_full   = np.load('./data/data_scaled_means.npy')
data_scaled_cosmos = np.load('./data/params_conc.npy')
data_scaled_cosmos_full = np.load('./data/params_conc_means.npy')

covariance = np.load('./data/covariance.npy')

In [4]:
print(data_scaled.shape,data_scaled_cosmos_full.shape)

(101, 9999, 50) (101, 3)


In [5]:
# fiducial parameters (for compression)
index   = 51
index_0 = 53
theta_fiducial = data_scaled_cosmos_full[index]
print(theta_fiducial)


[0.1 0.3 2.1]


In [6]:
theta_cov = data_scaled_cosmos_full[index_0]

### Covariance Matrix

In [7]:
# compute covariance from covariance sims
Cov_Inv = np.linalg.inv(covariance)

### Gaussian process interpolation of (precompressed) data

In [8]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C


kernel = C(1.0, (1e-4, 1e4)) * RBF(1, (1e-4, 1e4))
#Instanciate a Gaussian Process model
gp     = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)


# fit on averages of full rescaled set
gp.fit(data_scaled_cosmos_full,data_scaled_full)

#use the GP prediction to build the model
def fitGP(theta):
    pred, stdev = gp.predict(np.array(theta).reshape(1,3),return_std=True)
    return pred.T

# compute derivative of mean at fiducial model with finite differencing

h      = 0.01

theta1 = np.asarray([theta_fiducial[0]*(1+h), theta_fiducial[1],theta_fiducial[2]])
theta1_= np.asarray([theta_fiducial[0]*(1-h), theta_fiducial[1],theta_fiducial[2]])

theta2 = np.asarray([theta_fiducial[0], theta_fiducial[1]*(1+h),theta_fiducial[2]])
theta2_= np.asarray([theta_fiducial[0], theta_fiducial[1]*(1-h),theta_fiducial[2]])

theta3 = np.asarray([theta_fiducial[0], theta_fiducial[1],theta_fiducial[2]*(1+h)])
theta3_= np.asarray([theta_fiducial[0], theta_fiducial[1],theta_fiducial[2]*(1-h)])

dmudt1 = (fitGP(theta1)-fitGP(theta1_))/(theta1-theta1_)[0]
dmudt2 = (fitGP(theta2)-fitGP(theta2_))/(theta2-theta2_)[1]
dmudt3 = (fitGP(theta3)-fitGP(theta3_))/(theta3-theta3_)[2]

# derivative
dmudt = np.hstack((dmudt1,dmudt2,dmudt3)).T

In [9]:
# set up scrore compression
mu             = fitGP(theta_fiducial)[:,0]
Cinv           = Cov_Inv


Compressor     = score.Gaussian(len(mu), theta_fiducial, mu = mu, Cinv = Cinv, dmudt = dmudt)
Compressor.compute_fisher()
Finv           = Compressor.Finv

def compressor(d, compressor_args):
    return Compressor.scoreMLE(d)
compressor_args=None

/home/nessa/miniconda2/envs/py3torch/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:357: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [23]:
# compress all the (precompressed data) with compressor
compressed_train = np.zeros((data_scaled.shape[0],data_scaled.shape[1],3))
data_scaled_c    = np.zeros_like(compressed_train)
print(compressed_train.shape)
# compressed_train = np.reshape(compressed_train,newshape=(101,9999,3))
# data_scaled_c    = np.reshape(data_scaled_cosmos,newshape=(101,9999,3))
# data_scaled_= np.reshape(data_scaled,newshape=(101,9999,50))

# loop over cosmologies
for ii in range(compressed_train.shape[0]):
    for jj in range(compressed_train.shape[1]):
        compressed_train[ii][jj] = compressor(data_scaled[ii][jj],None)
        data_scaled_c[ii][jj] = data_scaled_cosmos_full[ii]
print(data_scaled_c.shape)

(101, 9999, 3)
(101, 9999, 3)


In [27]:
compressed_train1 = compressed_train[0:index,:,:]
print(compressed_train1.shape,compressed_train2.shape)
compressed_train2 = compressed_train[index+1:,:,:]
compressed_data   = compressed_train[index,:,:]
compressed_train_ = np.concatenate((compressed_train1,compressed_train2))
print(compressed_train_.shape, compressed_data.shape)

data_scaled_c1     = data_scaled_c[0:index,:,:]
data_scaled_c2     = data_scaled_c[index+1:,:,:]
data_scaled_c_data = data_scaled_c[index,:,:]
data_scaled_c_     = np.concatenate((data_scaled_c1,data_scaled_c2))
print(data_scaled_c_.shape, data_scaled_c_data.shape)

(51, 9999, 3) (49, 9999, 3)
(100, 9999, 3) (9999, 3)
(100, 9999, 3) (9999, 3)


In [28]:
compressed_train_ = np.reshape(compressed_train_,newshape=(-1,3))
data_scaled_c_    = np.reshape(data_scaled_c_,newshape=(-1,3))
print(data_scaled_c_.shape,compressed_train_.shape)

(999900, 3) (999900, 3)


### NDE estimation

In [29]:
# set up priors
lower = np.array([np.min(data_scaled_cosmos[:,0]),np.min(data_scaled_cosmos[:,1]),np.min(data_scaled_cosmos[:,2])])
upper = np.array([np.max(data_scaled_cosmos[:,0]),np.max(data_scaled_cosmos[:,1]),np.max(data_scaled_cosmos[:,2])])
print(upper, lower)
prior = priors.Uniform(lower, upper)

[0.62036 0.4159  2.9114 ] [0.     0.1841 1.2886]


In [30]:
# NDEs you wanna train
NDEs = [ndes.ConditionalMaskedAutoregressiveFlow(n_parameters=3, n_data=3, n_hiddens=[50,50], n_mades=5, act_fun=tf.tanh, index=0),
        ndes.ConditionalMaskedAutoregressiveFlow(n_parameters=3, n_data=3, n_hiddens=[50,50], n_mades=4, act_fun=tf.tanh, index=1),
        ndes.ConditionalMaskedAutoregressiveFlow(n_parameters=3, n_data=3, n_hiddens=[30,30], n_mades=5, act_fun=tf.tanh, index=2),
        ndes.ConditionalMaskedAutoregressiveFlow(n_parameters=3, n_data=3, n_hiddens=[30,30], n_mades=6, act_fun=tf.tanh, index=3)]

In [31]:
DelfiEnsemble = delfi.Delfi(compressed_data[0], prior, NDEs, 
                            Finv = Finv, 
                            theta_fiducial = theta_fiducial, 
                            param_limits = [lower, upper],
                            param_names = ['M_\nu', '\Omega_m', 'A_s'], 
                            results_dir = "./",
                            input_normalization="fisher")

In [32]:
DelfiEnsemble.load_simulations(compressed_train_,data_scaled_c_)

In [33]:
DelfiEnsemble.fisher_pretraining()

In [34]:
DelfiEnsemble.train_ndes()

KeyboardInterrupt: 

In [ ]:
posterior_samples = DelfiEnsemble.emcee_sample()

In [ ]:
DelfiEnsemble.triangle_plot(samples=[posterior_samples])

In [ ]:
samples = []
for i in range(4):
    samples.append(DelfiEnsemble.emcee_sample(log_likelihood=lambda x: DelfiEnsemble.log_posterior_individual(i, x)))

In [ ]:
for i in range(4):
    DelfiEnsemble.triangle_plot(samples=samples[i:i+1])